In [1]:
import pandas as pd
import numpy as np
import math
import operator

In [2]:
data = pd.read_excel("C:/Users/jiang/Desktop/课件/第六讲/order_data.xlsx")

In [3]:
data.head()

,user_id,product_id,catage_id,orders_num,buy_num,price
0,42,100104540,1127.0,1,1,22.00
1,42,100119749,1993.0,1,1,63.51
2,42,100145116,1127.0,2,2,61.00
3,42,100153506,1206.0,1,5,3.00
4,42,100157452,1104.0,1,2,12.50


In [4]:
data.dtypes

user_id         int64
product_id      int64
catage_id     float64
orders_num      int64
buy_num         int64
price         float64
dtype: object

In [5]:
data.isnull().any()

user_id       False
product_id    False
catage_id      True
orders_num    False
buy_num       False
price         False
dtype: bool

In [9]:
data = data.fillna(0)

In [10]:
data.head()

,user_id,product_id,catage_id,orders_num,buy_num,price
0,42,100104540,1127.0,1,1,22.00
1,42,100119749,1993.0,1,1,63.51
2,42,100145116,1127.0,2,2,61.00
3,42,100153506,1206.0,1,5,3.00
4,42,100157452,1104.0,1,2,12.50


In [6]:
data.isnull().any()

user_id       False
product_id    False
catage_id      True
orders_num    False
buy_num       False
price         False
dtype: bool

In [7]:
len(data)

107203

In [10]:
data = data[(data.product_id > 0)]
data["catage_id"] = data["catage_id"].apply(lambda x: int(x))

In [11]:
user_data = data.groupby("user_id").size()
user_data = user_data[user_data > 1]
data = data[data.user_id.isin(user_data.keys())]
user_list = data.values.tolist()

In [12]:
len(user_list)

76121

In [13]:
all_product_id = list(set(data["product_id"].values.tolist()))

In [14]:
len(all_product_id)

19462

In [46]:
#lst = [1,2,3,4,5,6]
#for index,value in enumerate(lst):
#     print ('%s,%s'%(index,value))

0,1
1,2
2,3
3,4
4,5
5,6


In [15]:
product_to_index = {}
index_to_product = {}
for index,value in enumerate(all_product_id):
    product_to_index[value] = index
    index_to_product[index] = value

In [17]:
user_item_index = {}
for user_id in user_data.keys():
    product_ids = data[data.user_id == user_id]["product_id"].values.tolist()
    for index,value in enumerate(product_ids):
        product_ids[index] = product_to_index[value]
    user_item_index[user_id] = product_ids

In [18]:
type(user_item_index)

dict

In [19]:
user_item_index

{42: [18649, 9291, 4443, 9801, 11744, 17414, 8798, 14410, 15869],
 165: [7992, 8099, 15013],
 172: [4722, 9949, 9984],
 204: [5968,
  2053,
  2232,
  2602,
  3093,
  3690,
  3825,
  3827,
  3832,
  4562,
  13118,
  16517,
  16530,
  16534,
  17574,
  17581,
  17585,
  17589,
  17591,
  17932,
  18216,
  18217,
  18237,
  18238,
  18239,
  18240,
  18241,
  11862,
  11863,
  13305,
  14355,
  118,
  8601],
 233: [11545, 11617],
 234: [1894, 1937],
 272: [14899, 781],
 416: [2045, 785, 17674, 9647],
 625: [18387,
  10520,
  8253,
  12243,
  13126,
  16116,
  3958,
  13877,
  14103,
  13772,
  2516,
  9890],
 789: [3704, 17792, 8181],
 808: [9291, 9834],
 831: [12787,
  14891,
  15427,
  1662,
  12128,
  2761,
  6756,
  6767,
  4489,
  15018,
  19431,
  1236,
  2242],
 1596: [5073, 6531, 13118, 14099],
 3330: [15882, 9442],
 5143: [18362, 411, 11159],
 5665: [11105, 11283, 12787, 4524],
 5746: [16209, 16326],
 5816: [16357,
  16877,
  16956,
  4475,
  2195,
  2667,
  5513,
  5571,
  10784

In [20]:
product_length = len(product_to_index)
matrix_c = np.zeros((product_length,product_length))

In [21]:
matrix_c

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
for user_id in user_item_index:
    product_ids = user_item_index[user_id]
    for i,value in enumerate(product_ids):
        if(i < len(product_ids) - 1):
            list_other = product_ids[(i+1):len(product_ids)]
            for second_product_index in list_other:
                matrix_c[value][second_product_index] += 1
                matrix_c[second_product_index][value] += 1

In [26]:
product_index_count_dic = {}
product_group = data.groupby("product_id").size()
for product_id in product_group.keys():
    product_index_count_dic[product_to_index[product_id]] = product_group[product_id]
matrix_w = np.zeros((product_length,product_length))

In [35]:
index_i_list,index_j_list = np.where(matrix_c > 0)
for index,value in enumerate(index_i_list):
    i = value
    j = index_j_list[index]
    score = matrix_c[i][j]/math.sqrt(product_index_count_dic[i] * product_index_count_dic[j])
    matrix_w[i][j] = score
    matrix_w[j][i] = score

In [38]:
matrix_w[i][j]

0.7071067811865475

In [61]:
a = np.zeros(product_length)
a[1] = 3
a[2] = 4
a[5] = 6
a = (a - np.min(a))/(np.max(a) - np.min(a))

In [62]:
a

array([0.        , 0.5       , 0.66666667, ..., 0.        , 0.        ,
       0.        ])

In [42]:
def normalize(value):
    value = (value - np.min(value))/(np.max(value) - np.min(value))
    return value

In [43]:
user_like_item_dic = {}
for user_id in user_data.keys():
    user_like_item = data[data.user_id == user_id]
    user_item_like_matrix = np.zeros(product_length)
    for i in range(len(user_like_item)):
        index = product_to_index[user_like_item.iloc[i].product_id]
        user_item_like_matrix[index] = user_like_item.iloc[i].orders_num
    user_like_item_dic[user_id] = normalize(user_item_like_matrix)

In [44]:
user_like_item_dic[42]

array([0., 0., 0., ..., 0., 0., 0.])

In [45]:
def getMostSimilar(matrix_w,index,k):
    c_list = matrix_w[index]
    similar_item = pd.DataFrame({"value":c_list})
    similar_item = similar_item.sort_values(by="value",ascending=False).iloc[0:k]
    similar_item_dic = {}
    for i in range(len(similar_item)):
        similar_item_dic[similar_item.iloc[i].name] = similar_item.iloc[i].value
    return similar_item_dic

In [53]:
getMostSimilar(matrix_w,0,10)

{15335: 0.7071067811865475,
 16418: 0.5773502691896258,
 2095: 0.3779644730092272,
 18043: 0.08084520834544433,
 0: 0.0,
 12971: 0.0,
 12976: 0.0,
 12975: 0.0,
 12974: 0.0,
 12973: 0.0}

In [54]:
len(user_like_item_dic)

18520

In [57]:
def reommendItem(user_id,matrix_w,user_like_item_dic,k):
    recommend_dic = {}
    user_like_list = user_like_item_dic[user_id]
    user_like_item_index_list = np.where(user_like_list > 0)
    user_like_item_index_list = user_like_item_index_list[0]
    for product_index in user_like_item_index_list:
        like_score = user_like_list[product_index]
        most_similar_item = getMostSimilar(matrix_w,product_index,k)
        for key in most_similar_item.keys():
            if key in user_like_item_index_list:
                continue
            #最终得分是用户对商品的喜欢程度 * 商品的相似程度
            score = like_score * most_similar_item[key]
            if key in recommend_dic.keys():
                score += recommend_dic[key]
            recommend_dic[key] = score
    #返回得分最高的k个商品
    sorted_x = sorted(recommend_dic.items(), key=operator.itemgetter(1))
    sorted_x.reverse()
    return sorted_x[0:k]

In [58]:
recommend_dic = reommendItem(42,matrix_w,user_like_item_dic,10)
recommend_dic

[(7762, 0.35355339059327373),
 (7271, 0.25),
 (9833, 0.25),
 (15951, 0.20952908873087345),
 (12536, 0.20412414523193154),
 (11433, 0.15811388300841897),
 (13956, 0.15617376188860607),
 (7871, 0.15617376188860607),
 (8195, 0.15617376188860607),
 (4715, 0.15617376188860607)]

In [59]:
def getAllUserRecommend():
    user_recommend = {}
    for user_id in user_like_item_dic.keys():
        #print(user_id)
        recommend_dic = reommendItem(user_id,matrix_w,user_like_item_dic,10)
        value = ""
        for key in recommend_dic:
            index = key[0]
            if value == "":
                value += str(index_to_product[index])
            else:
                value += "," + str(index_to_product[index])
        user_recommend[user_id] = value
    return user_recommend

In [60]:
res = getAllUserRecommend()

In [61]:
print(res)

{42: '100182858,100280368,100153593,100230199,100322431,100156977,100324776,100183013,100183589,100145434', 165: '100515582,100494455,100246192,100324926,100490775,100489700,100324147,100509744,100247916,100324751', 172: '100141583,100495211,100229407,100256245,100020321,100497524,100193491,100295459,100462355,100469891', 204: '100505162,100466505,100201006,100496679,100512438,100496742,100480008,100505375,100280157,100168529', 233: '100068540,100177651,100182363,100156348,100279305,100468556,100079077,100158495,100218527,100159151', 234: '100331513,100248512,100079436,100207117,100341966,100495849,100480927,100455150,100200746,100511354', 272: '100163043,100168403,100174112,100448513,100147905,100176514,100158556,100470475,100142780,100174151', 416: '100474174,100142956,100341280,100076832,100142568,100509900,100157147,100323862,100422280,100481984', 625: '100260365,100257254,100273016,100285397,100068828,100440058,100490439,100165549,100514314,100517220', 789: '100391160,100168134,10